<h2>Term Implementations</h2>

In [32]:
#deepcopy used to create repeated terms while keeping the original (that is possibly used elsewhere) intact.
from copy import deepcopy

#Registry of created terms used by enumeration algorithm.
fullList = {'constants': [], 'variables': [], 'functions': [], 'predicates': [], 'clauses': []}


#Implementation of contant. Overrides str, len, eq, and hash.
class Constant:
    registry = fullList['constants']
    
    def __init__(self, s):
        self.sym = s
        self.arity = 0
        self.length = 1
        Constant.registry.append(self)
        
    def __str__(self):
        return self.sym
        
    def __len__(self):
        return self.length
    
    def __eq__(self, other):
        return isinstance(other, Constant) and self.sym == other.sym

    def __hash__(self):
        return id(self)
    
    #Tested different naming scheme. Not currently in use.
    def name(self,s):
        if sum([1 if i.sym==s else 0 for i in Constant.registry])>0:
            return self.name(s + ".")
        else: return s
    
    #Constant size can not be reduced. Higher-level structures can determine if it is a repeat.
    def reduceSize(self):
        pass
    
    #Will always be a ground term. Base of higher level isGround recursion.
    def isGround(self):
        return True
    
    #No variables. Returns empty list to conform with other structures use of list concacts.
    def findVariables(self):
        return []
    
    #Is Inherently a repeated term once found. Base of higher level findRepeats recursion.
    def findRepeats(self):
        return [self]
    
    #Nothing to replace. Base of higher level replace recursion.
    def replace(self, var, rpt):
        pass

#Implementation of repeat. Encapsulates replaced object in case the information is needed. Length is always 0.
#Overrides str, len, eq, and hash
class Repeat:
    def __init__(self, original):
        self.sym = '*'
        self.origin = original
        self.length = 0
    
    #Currently shows just a star. Switch commented line for more informative printed information.
    def __str__(self):
        #return self.sym + str(self.origin) + self.sym
        return self.sym
        
    def __len__(self):
        return self.length
    
    #Checks encapsulated object of equality beyond just being a repeat.
    def __eq__(self, other):
        return isinstance(other, type(self.origin)) and \
        self.origin.sym == other.sym and self.origin.params == other.params
        
    def __hash__(self):
        return id(self)
    
    #Size already reduced. Implemented so that reduceSize still works if accidentally called multiple times.
    def reduceSize(self):
        pass
    
    #Ground determined by encapsulated object. Passed on to that function.
    def isGround(self):
        return self.origin.isGround()
    
    #Variables determined by encapsulated object. Passed on to that function.
    def findVariables(self):
        return self.origin.findVariables()
    
    #First instance of this repeat already included in search. 
    #Empty list used to conform with higher level findRepeat.
    def findRepeats(self):
        return []
    
    #No variable to replace. Should not replace underlying object (if possible) 
    #because there is some clause that does not have this as a repeat.
    def replace(self, var, rpt):
        pass

#Implementaion of variable. Frequently replaced with other objects in enumerate algorithm.
#Overrides str, len, eq, and hash.
class Variable:
    registry = fullList['variables']
    
    def __init__(self, s):
        self.sym = s
        self.length = 1 #Use minimal possible size.
        #self.length = 0
        Variable.registry.append(self)
        
    def __str__(self):
        return self.sym
    
    def __len__(self):
        return self.length
    
    def __eq__(self, other):
        return isinstance(other, Variable) and self.sym == other.sym
    
    def __hash__(self):
        return id(self)
    
    #Previous naming scheme testing. Not in use.
    def name(self, s):
        if sum([1 if i.sym==s else 0 for i in Variable.registry])>0:
            return self.name(s + ".")
        else: return s
    
    #Variable size can not be reduced. Higher-level structures can determine if it is a repeat.
    def reduceSize(self):
        pass
    
    #Will never be a ground term. Base of higher level isGround recursion.
    def isGround(self):
        return False
    #Is a variable. Returns object wrapped in list to conform with other structures use of list concacts.
    #Base of higher level findVariables recursion.
    def findVariables(self):
        return [self]
    
    #Is Inherently a repeated term once found. Base of higher level findRepeats recursion.
    def findRepeats(self):
        return [self]
    
    def replace(self, var, rpt):
        variable = deepcopy(self)
        if self==var:
                return rpt
        else:
                return

#Implementation of function. Includes symbol (must be unique) and list of paramaters.
#Registry containts copies of all functions that may be altered in enumerate algorithm.
#Consider including arity parameter.
#Overrides str, len, eq, and hash
class Function:
    registry = fullList['functions']
    
    #Length calculated by definition: 1 + length of each parameter.
    def __init__(self, s, *args):
        self.sym = s
        self.params = list(args)
        self.arity = len(args)
        self.length = 1+sum([len(i) for i in self.params])
        Function.registry.append(deepcopy(self))

    def __str__(self):
        toReturn = self.sym + "("
        for i in self.params:
            toReturn += str(i) + ","
        toReturn = toReturn[:-1] + ")"
        return toReturn
    
    def __len__(self):
        return 1+sum([len(i) for i in self.params])
                     
    def __eq__(self, other):
        return isinstance(other, Function) and self.sym == other.sym and self.params == other.params
    
    def __hash__(self):
        return id(self)
    
    #Previous naming scheme testing. Not in use.
    def name(self,s):
        if sum([1 if i.sym==s else 0 for i in Function.registry])>0:
            return self.name(s + ".")
        else: return s
    
    #For each parameter (0 to arity), scans forward for repeated terms. Any repeats are replaced by Repeats.
    #Then reduces size of each parameter.
    def reduceSize(self):
        for i in xrange(len(self.params)):
            for j in xrange(i+1, len(self.params)):
                if not isinstance(self.params[j], Repeat) and self.params[i] == self.params[j]:
                    self.params[j] = Repeat(self.params[j])
        for i in self.params:
            i.reduceSize()
    
    #Scans each parameter. If any are not ground, the entire instance of the function is not ground.
    def isGround(self):
        for i in self.params:
            if not i.isGround():
                return False
        return True
    
    #Scans each parameter. Finds variables in each parameter and adds all of them to the list.
    def findVariables(self):
        variables = []
        for i in self.params:
            variables = variables + i.findVariables()
        return variables
    
    #Scans each parameter. Finds repeats in each parameter and adds all of them to the list.
    def findRepeats(self):
        repeats = []
        for i in self.params:
            repeats = repeats + i.findRepeats()
        repeats.append(self)
        return repeats
    
    def replace(self, var, rpt):
        function = deepcopy(self)
        for i in function.params:
            if i==var:
                function.params[function.params.index(var)] = rpt
            else:
                i.replace(var, rpt)
        return function

#Implementation of predicate (very similar implementation to function). 
#Includes symbol (must be unique) and list of paramaters.
#Registry containts copies of all functions that may be altered in enumerate algorithm.
#Overrides str, len, eq, and hash
class Predicate:
    registry = fullList['predicates']
    
    #Length calculated by definition: 1 + length of each parameter.
    def __init__(self, s, *args):
        self.sym = s
        self.params = list(args)
        self.length = 1+sum([len(i) for i in self.params])
        Predicate.registry.append(deepcopy(self))
        
    def __str__(self):
        toReturn = self.sym + "("
        for i in self.params:
            toReturn += str(i) + ","
        toReturn = toReturn[:-1] + ")"
        return toReturn
    
    def __len__(self):
        return 1+sum([len(i) for i in self.params])
    
    def __eq__(self, other):
        return isinstance(other, Predicate) and self.params == other.params
 
    def __hash__(self):
        return id(self)
    
    #Previous naming scheme testing. Not in use.
    def name(self,s):
        if sum([1 if i.sym==s else 0 for i in Function.registry])>0:
            return self.name(s + ".")
        else: return s
    
    #For each parameter (0 to arity), scans forward for repeated terms. Any repeats are replaced by Repeats.
    #Then reduces size of each parameter.
    def reduceSize(self):
        for i in xrange(len(self.params)):
            for j in xrange(i+1, len(self.params)):
                if not isinstance(self.params[j], Repeat) and self.params[i] == self.params[j]:
                    self.params[j] = Repeat(self.params[j])
        for i in self.params:
            i.reduceSize()
            
    #Scans each parameter. If any are not ground, the entire instance of the function is not ground.
    def isGround(self):
        for i in self.params:
            if not i.isGround():
                return False
        return True
    
    #Scans each parameter. Finds variables in each parameter and adds all of them to the list.
    def findVariables(self):
        variables = []
        for i in self.params:
            variables = variables + i.findVariables()
        return variables
    
    #Scans each parameter. Finds repeats in each parameter and adds all of them to the list.
    def findRepeats(self):
        repeats = []
        for i in self.params:
            repeats = repeats + i.findRepeats()
        repeats.append(self)
        return repeats

    def replace(self, var, rpt):
        predicate = deepcopy(self)
        for i in predicate.params:
            if i==var:
                predicate.params[predicate.params.index(var)] = rpt
            else:
                i.replace(var, rpt)
        return predicate

#Implementation of clause (very similar implementation to function and predicate). 
#Includes list of paramaters.
#Registry containts copies of all functions that may be altered in enumerate algorithm.
#Overrides str, len, eq, and hash
class Clause:
    registry = fullList['clauses']
    
    #Length calculated by definition: sum of length of each parameter.
    def __init__(self, *args):
        self.params = list(args)
        self.length = sum([len(i) for i in self.params])
        Clause.registry.append(deepcopy(self))
        
    def __str__(self):
        toReturn = "{"
        for i in self.params:
            toReturn += str(i) + ","
        toReturn = toReturn[:-1] + "}"
        return toReturn
    
    def __len__(self):
        return sum([len(i) for i in self.params])
    
    def __eq__(self, other):
        return isinstance(other, Function) and self.sym == other.sym and self.params == other.params

    def __hash__(self):
        return id(self)
    
    #For each parameter (0 to arity), scans forward for repeated terms. Any repeats are replaced by Repeats.
    #Then reduces size of each parameter. Returns new modified clause.
    def reduceSize(self):
        for i in xrange(len(self.params)):
            for j in xrange(i+1, len(self.params)):
                if not isinstance(self.params[j], Repeat) and self.params[i] == self.params[j]:
                    self.params[j] = Repeat(self.params[j])
        for i in self.params:
            i.reduceSize()    
        return self
    
    #Scans each parameter. If any are not ground, the entire instance of the function is not ground.
    def isGround(self):
        for i in self.params:
            if not i.isGround():
                return False
        return True
    
    #Scans each parameter. Finds variables in each parameter and adds all of them to the list.
    def findVariables(self):
        variables = []
        for i in self.params:
            variables = variables + i.findVariables()
        return variables
    
    #Scans each parameter. Finds repeats in each parameter and adds all of them to the list.
    def findRepeats(self):
        repeats = []
        for i in self.params:
            repeats = repeats + i.findRepeats()
        repeats.append(self)
        return repeats
    
    def replace(self, var, rpt):
        clause = deepcopy(self)
        for i in clause.params:
            if i==var:
                clause.params[clause.params.index(var)] = rpt
            else:
                i.replace(var, rpt)
        return clause
    
print "No syntax errors in classes."

No syntax errors in classes.


<h2>Testing Zone</h2>
Replace is not currently working because of the way listcomps work in python, Cant change values in the list using a 'for i in list' approach. Working on fix.

In [34]:
#reduceSize() could potentially be done in linear time by implementing a trie datastructure for searching. 
#Since clauses are generally relatively small, this n^2 methods should be efficient enough.
#Potentially come back to make more efficient.
#TODO: Look at objects in different higher-level objects
            
a = Constant('a')
b = Constant('a')
#print str(a), str(b)
#print fullList['constants']
x = Variable('x')
y = Variable('y')
a1 = Repeat(a)
f1 = Function('f', a, x)
f2 = Function('f', a, a)
g1 = Function('g', b, x)
g2 = Function('g', b, y)
g3 = Function('g', b, y)

p1 = Predicate('P',f1, f2, g2, g3)
p2 = Predicate('Q', f1)
p3 = Predicate('R', f2, g3)
p4 = Predicate('S', x, a)

c1 = Clause(p1, p2, p3)
c2 = Clause(f1)


#print str(f1) + " is ground: " + str(f1.isGround())
#print str(f2) + " is ground: " + str(f2.isGround())
#print
#print "Original Clause:" + str(c1)
#print "Original Length: " + str(len(c1))
#print
#c1.reduceSize()
#print "Modified Clause:" + str(c1)
#print "Modified Length: " + str(len(c1))
#for i in set(c1.findRepeats()):
#    print i
    
clause = c1

print p4
cz = p4.replace(x, a)
print cz

S(x,a)
S(a,a)


In [10]:
constants = Constant.registry
functions = Function.registry
globalRepeats = set()
clauseSet = set()

SIZE_BOUND = 3

#repeats
def enumerateClauses(clause, size, rpts):
    global constants, functions
    repeats = rpts.union(set(clause.findRepeats()))
    if size<0:
        return
    elif size==0:
        x = None
        for x in clause.findVariables():
            for i in repeats:
                clauseSet.add(clause.replace(x,i))
        if x==None:
            clauseSet.add(clause)
    else:
        for x in clause.findVariables():
            for i in set(constants).union(set(functions)).union(set(repeats)):
                if size-len(i)>=0:
                    enumerateClauses(clause.replace(x,i), size-len(i), repeats)
        sizeCheck(SIZE_BOUND, repeats)
        
def sizeCheck(size, repeats):
    for clause in clauseSet:
        if len(clause.reduceSize()) > size:
            clauseSet.remove(clause)
        else:
            for i in clause.params:
                repeats.add(i)

clauseSet.add(c2)
enumerateClauses(c2, SIZE_BOUND, globalRepeats)
for i in clauseSet:
    print i

{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{f(a,x)}
{

In [12]:
a = [1,2,3]
b = a
a = [1,2,4]
print b

a = a + []
print a

a = Constant('a')
l = [a,b]
print a==l[0]

b = Constant('b')
x = Variable('x')
y = Variable('y')
a1 = Repeat(a)
f1 = Function('f', a, x)

print f1.params
f1.params[f1.params.index(a)] = b
print f1.params

print a
print b
b = deepcopy(a)

print a
print b
print a==b

print set(constants).union(set(functions))

[1, 2, 3]
[1, 2, 4]
True
[<__main__.Constant instance at 0x104244098>, <__main__.Variable instance at 0x103a993b0>]
[<__main__.Constant instance at 0x104065d40>, <__main__.Variable instance at 0x103a993b0>]
a
b
a
a
True
set([<__main__.Constant instance at 0x104065d40>, <__main__.Function instance at 0x1042e5200>, <__main__.Function instance at 0x1040b8b90>, <__main__.Constant instance at 0x1040a96c8>, <__main__.Function instance at 0x1042e5368>, <__main__.Function instance at 0x1040b96c8>, <__main__.Constant instance at 0x1042e2f38>, <__main__.Constant instance at 0x1040e0830>, <__main__.Constant instance at 0x1042e2ea8>, <__main__.Function instance at 0x1042e52d8>, <__main__.Function instance at 0x1042e53f8>, <__main__.Function instance at 0x1042733f8>, <__main__.Constant instance at 0x104244098>])
